In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import time
import os
import copy

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/vit/models/01-24-2024 19-25-41/'
MODEL_LAST_EPOCH = 'epoch_8.pth'

# CONFIG VARIABLES
DATA_DIR = 'E:\Datasets\cor-splits\sgkf-8-1-1-4000'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 25
LR_EPOCH_SCHEDULE = 7
LR_GAMMA = 0.1

In [4]:
working_folder = 'notebooks/vit'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
# curr_time = datetime(curr_time.year, curr_time.month, curr_time.day, curr_time.hour)
curr_time = curr_time.strftime("%Y-%d-%m %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
# vit_b_16
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [6]:
data_folders = ['train', 'val']

batch_size = 64
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x),
                                          data_transforms[x])
                  for x in data_folders}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=0)
              for x in data_folders}

dataset_sizes = {x: len(image_datasets[x]) for x in data_folders}

class_names = image_datasets['train'].classes
print(class_names)

['0', '1']


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [15]:
def train_model(model, criterion, optimizer, scheduler, output_folder, start_epoch=1, num_epochs=25, epoch_save_interval=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    log_path = f"{output_folder}/log.txt"
    
    run_info = 'Dataset {}\nLearning Rate Epoch Schedule = {}\nLearning Rate Gamma = {}'.format(DATA_DIR, LR_EPOCH_SCHEDULE, LR_GAMMA)
            
    if not os.path.exists(log_path): 
        log = open(log_path,'x')
        log.writelines('=' * 10+'\n')
        log.writelines('\n'+run_info+'\n')
        log.writelines('=' * 10+'\n')
        log.close()        
        
    for epoch in range(start_epoch, num_epochs+1):
        epoch_start = time.time()
        
        epoch_info = 'Epoch {}/{}'.format(epoch, num_epochs)
        
        with open(log_path,'a') as log:
            log.writelines('\n'+epoch_info+'\n')
            log.writelines('-' * 10+'\n')
        
        print(epoch_info)
        print('-' * 10)
        

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            loss_acc_info = '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)
            with open(log_path,'a') as log:
                log.writelines(loss_acc_info+'\n')
            print(loss_acc_info)
            
            if epoch % epoch_save_interval == 0:
                torch.save(model.state_dict(), f'{output_folder}/epoch_{epoch}.pth')
                
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        epoch_end = time.time() - epoch_start
        epoch_duration_info = 'Epoch duration: {:.0f} m {:.0f}s'.format(epoch_end//60, epoch_end%60)
        with open(log_path,'a') as log:
            log.writelines(epoch_duration_info+'\n')

        print(epoch_duration_info)
        print()
    
    torch.save(model.state_dict(), f'{output_folder}/last.pth')
    
    time_elapsed = time.time() - since
    
    complete_text = 'Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)
    print(complete_text)
    best_acc_text = 'Best val Acc: {:4f}'.format(best_acc)
    print(best_acc_text)
    
    with open(log_path,'a') as log:
        log.writelines(complete_text+'\n')
        log.writelines(best_acc_text+'\n')
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), f'{output_folder}/best.pth')
    
    return model

In [16]:
model = models.vit_b_16(pretrained=True)
# model = models.vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = False

# A newly defined layer is created with requires_grad=True by default
model.heads = nn.Sequential(
    nn.Linear(in_features=768, out_features=2)
    )

if CONTINUE_TRAINING:
    output_folder = MODEL_WEIGHTS_PATH
    model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
else:
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)
             
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
optimizer = optim.SGD(model.heads.parameters(), lr=0.001)
# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)

model = train_model(model, criterion, optimizer, step_lr_scheduler, output_folder, start_epoch=START_EPOCH, num_epochs=N_EPOCHS)

c:\Users\Adm\anaconda3\envs\cv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Adm\anaconda3\envs\cv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25
----------
train Loss: 0.6866 Acc: 0.5740
val Loss: 0.6628 Acc: 0.5800
Epoch duration: 3 m 42s

Epoch 2/25
----------
train Loss: 0.5794 Acc: 0.7140
val Loss: 0.6442 Acc: 0.6026
Epoch duration: 2 m 36s

Epoch 3/25
----------
train Loss: 0.5282 Acc: 0.7578
val Loss: 0.6439 Acc: 0.6026
Epoch duration: 2 m 27s

Epoch 4/25
----------
train Loss: 0.4941 Acc: 0.7790
val Loss: 0.6487 Acc: 0.6110
Epoch duration: 2 m 10s

Epoch 5/25
----------
train Loss: 0.4680 Acc: 0.7946
val Loss: 0.6675 Acc: 0.6134
Epoch duration: 2 m 5s

Epoch 6/25
----------
train Loss: 0.4467 Acc: 0.8091
val Loss: 0.6582 Acc: 0.5967
Epoch duration: 2 m 6s

Epoch 7/25
----------
train Loss: 0.4295 Acc: 0.8223
val Loss: 0.6627 Acc: 0.5990
Epoch duration: 2 m 10s

Epoch 8/25
----------
train Loss: 0.4198 Acc: 0.8258
val Loss: 0.6641 Acc: 0.5979
Epoch duration: 2 m 19s

Epoch 9/25
----------
train Loss: 0.4179 Acc: 0.8286
val Loss: 0.6648 Acc: 0.5967
Epoch duration: 2 m 13s

Epoch 10/25
----------
train Loss: 0.41